# Incompressible fluids

The Navier-Stokes equations for an incompressible fluid are 

$$
\frac{\partial \vec{u}}{\partial t} + (\vec{u} \cdot \nabla) \vec{u} = -\frac{1}{\rho} \nabla p 
+ \nu \nabla ^2 \vec{u} \\ 
\nabla \cdot \vec{u} = 0
$$

Where $ \vec{u} = (u,v,w) $ is a velocity field, $p$ is a scalar pressure field, $\rho$ is fluid density, and $\nu$ is the kinematic viscosity. The first equation represents the volume conservation, which is equivalent to mass conservation for an incompressible fluid. For a 2D system, we can write the first equation as 

$$
\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y} =
-\frac{1}{\rho}\left(\frac{\partial p}{\partial x}\right) + \nu \left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right)
$$

$$
\frac{\partial v}{\partial t} + u\frac{\partial v}{\partial x} + v \frac{\partial v}{\partial y} =
-\frac{1}{\rho}\left(\frac{\partial p}{\partial y}\right) + \nu \left( \frac{\partial^2 v}{\partial x^2} + \frac{\partial^2 v}{\partial y^2} \right)
$$


The second equation represents the conservation of momentum in our system, given by a force balance. To numerically solve the momentum equation, we choose the following time scheme:

$$
\frac{\left(\vec{u}^{n+1} - \vec{u}^{n + \frac{1}{2}}\right)
- \left(\vec{u}^{n+\frac{1}{2}} - \vec{u}^n \right)
}{\Delta t}
+ \vec{u}^n \cdot \nabla \vec{u}^n = -\frac{1}{\rho} \nabla p^{n + \frac{1}{2}} + \nu \nabla^2\vec{u}^n
$$

We can decouple pressure and velocity fields as

$$
\frac{\vec{u}^{n + \frac{1}{2}} - \vec{u}^n}{\Delta t} + \vec{u}^n \cdot \nabla \vec{u}^n = \nu \nabla^2 \vec{u}^n
$$
$$
\frac{\vec{u}^{n+1} - \vec{u}^{n + \frac{1}{2}}}{\Delta t} = - \frac{1}{\rho}\nabla p^{n + \frac{1}{2}}
$$

Taking the divergence of the pressure equation, and forcing $\nabla \cdot \vec{u}^{n + 1} = 0~$ in order to ensure continuity, we obtain a Poisson equation for pressure.

$$
\nabla^2 p^{n + \frac{1}{2}} = \rho \frac{\nabla \cdot \vec{u}^{n+\frac{1}{2}}}{\Delta t}
$$

We can iteratively solve for $p^{n+\frac{1}{2}}~$ using the Gauss-Seidel method, and then advance the velocity field from $\vec{u}^{n+\frac{1}{2}}~$ to $\vec{u}^{n+1}$

# Numerical Scheme

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

**(1) Calculate advection terms using 1st order upwind scheme **

$$u \frac{\partial u}{\partial x} = u_{i,j} \frac{u_{i,j} - u_{i,j-1}}{\Delta x} ~~\text{for}~~ u_{i,j} > 0$$

$$u \frac{\partial u}{\partial x} = u_{i,j} \frac{u_{i,j+1} - u_{i,j}}{\Delta x} ~~\text{for}~~ u_{i,j} < 0$$

In [2]:
def advec_terms_2D(nx,ny,dx,dy,u,v,vel):
    """
    Calculates advection terms of Navier-Stokes equation using 1st order upwind scheme
    
    nx,ny = number of grid points in x and y directions
    dx,dy = lattice spacing in x and y directions 
    u,v = x and y components of velocity field
    vel = component u or v of velocity field for which you are solving NS equation 
    """
    advec_x = np.zeros((ny,nx))
    advec_y = np.zeros((ny,nx))
    advec_2D = np.zeros((ny,nx))
    
    for i in range(1,ny-1):
        for j in range(1,nx-1):
            
            if u[i,j] > 0:
                advec_x[i,j] = u[i,j] * (u[i,j] - u[i,j-1])/dx
            else:
                advec_x[i,j] = u[i,j] * (u[i,j+1] - u[i,j])/dx
                
            if v[i,j] > 0:
                advec_y[i,j] = v[i,j] * (v[i,j] - v[i-1,j])/dy
            else:
                advec_y[i,j] = v[i,j] * (v[i+1,j] - v[i,j])/dy
                
    advec_2D[1:ny-1,1:nx-1] = advec_x[1:ny-1,1:nx-1] + advec_y[1:ny-1,1:nx-1]
            
    return(advec_2D)

** (2) Calculate derivatives using 1st order central differencing **

$$\frac{\partial p}{\partial x} = \frac{p_{i,j+1} - p_{i,j-1} }{2\Delta x} $$

In [3]:
def gradient_2D(nx,ny,dx,dy,f):
    """
    Calculates 1st derivatives using central difference 
    
    nx,ny = number of grid points in x and y directions
    dx,dy = lattice spacing in x and y directions
    f = argument to take gradient of   
    """
    
    dfdx = np.zeros((ny,nx))
    dfdy = np.zeros((ny,nx))
    
    dfdx[1:ny-1,1:nx-1] = (f[1:ny-1,2:nx] - f[1:ny-1,0:nx-2])/(2*dx)
    dfdy[1:ny-1,1:nx-1] = (f[2:ny,1:nx-1] - f[0:ny-2,1:nx-1])/(2*dy)
    
    return(dfdx,dfdy) 

** (3) Calculate laplacians using 2nd order central differencing **

$$\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = \frac{u_{i,j-1} - 2 u_{i,j} + u_{i,j+1}}{\Delta x^2} + \frac{u_{i-1,j} - 2 u_{i,j} + u_{i+1,j}}{\Delta y^2}$$


In [4]:
def laplacian_2D(nx,ny,dx,dy,f):
    """
    Calculates laplacian using 2nd order central difference 
    
    nx,ny = number of grid points in x and y directions
    dx,dy = lattice spacing in x and y directions
    f = argument to take laplacian of 
    """
    
    lap = np.zeros((ny,nx))
    
    lap[1:ny-1,1:nx-1] = (f[1:ny-1,0:nx-2] - 2*f[1:ny-1,1:nx-1] + f[1:ny-1,2:nx])/(dx**2) + \
                         (f[0:ny-2,1:nx-1] - 2*f[1:ny-1,1:nx-1] + f[2:ny,1:nx-1])/(dy**2)
        
    return(lap)

**(4) Calculate the right-hand side of pressure poisson equation **

$$
b_{i,j} = \frac{\rho}{\Delta t} \left(\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y}\right)
$$

In [5]:
def RHS_poisson(nx,ny,dx,dy,rho,dt,u,v):
    """
    Calculates right hand side of pressure-poisson equation 
    
    nx,ny = number of grid points in x and y directions
    dx,dy = lattice spacing in x and y directions
    rho = density
    dt = timestep
    u,v = x and y components of velocity field 
    """
    
    dudx,dudy = gradient_2D(nx,ny,dx,dy,u)
    dvdx,dvdy = gradient_2D(nx,ny,dx,dy,v)
    
    b = (rho/dt)*(dudx + dvdy)
    
    return(b)

**(5) Use Gauss-Seidel SOR method to iteratively solve poisson equation for pressure**

The Gauss-Seidel method is an iterative method for solving systems of linear equations. It is similar to the Jacobi relaxation method, but has the advantage of referencing already updated points each time it loops through the array. The order in which we go through the elements doesn't matter, so we can choose to do so in a Red-Black "checkerboard" pattern in order to maximize the amount of updated points available. In addition, execution time can further be reduced via *over-relaxation*, in which we over-correct our system in order to reach convergence faster. [More on iterative methods for linear systems](https://math.berkeley.edu/~wilken/228A.F07/chr_lecture.pdf)

We advance our pressure in pseudo-time $m$ according to the following scheme until convergence:

$$
p_{i,j}^{m+1} = p_{i,j}^m + \omega \left(\frac{b_{i,j} - \sigma}{a} - p_{i,j}^m\right) \\
$$

where 

$$ 
\sigma = \frac{1}{{\Delta x}^2}\left(p_{i,j-1} + p_{i,j+1}\right) + \frac{1}{{\Delta y}^2}\left(p_{i-1,j} + p_{i+1,j}\right) 
\hspace{1cm}
a = {-2}\left(\frac{1}{{\Delta x}^2} + \frac{1}{{\Delta y}^2}\right)
\hspace{1cm}
\omega = \frac{2}{1 + \sin\left(\frac{2\pi}{n}\right)}
$$

In [6]:
def solve_poisson_2D(nx,ny,dx,dy,b,p,tol):
    """
    Solves poisson equation for pressure using successive over-relaxation method
    
    nx,ny = number of grid points in x and y direction
    dx,dy = lattice spacing in x and y directions
    b = right hand side of pressure poisson equation
    p = initial guess for pressure profile
    tol = convergence criteria
    """
    
    resd = np.zeros((ny,nx))
    tmp = np.zeros((ny,nx))
    
    nrm = 1.0
    omega = 2/(1 + np.sin(np.pi*dx))
    a = -2*(1/dx**2 + 1/dy**2)
    
    # Red-black ordering 
    
    b1_top = 1 # Start with black top left corner
    b1_left = 1
    b2_top = 2
    b2_left = 2
    
    r1_top = 1
    r1_left = 2
    r2_top = 2
    r2_left = 1
    
    if nx % 2 == 0:     # even number of columns
        
        b1_right = nx - 2
        b2_right = nx - 1
        r1_right = nx - 1
        r2_right = nx - 2
        
    else:               # odd number of columns
        
        b1_right = nx - 1
        b2_right = nx - 2
        r1_right = nx - 2
        r2_right = nx - 1
        
    if ny % 2 == 0:     # even number of rows
        
        b1_bottom = ny - 2
        b2_bottom = ny - 1
        r1_bottom = ny - 2
        r2_bottom = ny - 1
        
    else:               # odd number of rows
        
        b1_bottom = ny - 1
        b2_bottom = ny - 2
        r1_bottom = ny - 1
        r2_bottom = ny - 2

        
    niter = 0
    while nrm > tol:
        
        # Loop though pressure array in red-black order 
        
        for i in range(b1_top,b1_bottom+1,2):
            for j in range(b1_left,b1_right+1,2):
                # SOR method
                sigma = (p[i,j-1] + p[i,j+1])/dx**2 + (p[i-1,j] + p[i+1,j])/dy**2
                p[i,j] = p[i,j] + omega*((b[i,j] - sigma)/a - p[i,j])
                
        for i in range(b2_top,b2_bottom+1,2):
            for j in range(b2_left,b2_right+1,2):
                sigma = (p[i,j-1] + p[i,j+1])/dx**2 + (p[i-1,j] + p[i+1,j])/dy**2
                p[i,j] = p[i,j] + omega*((b[i,j] - sigma)/a - p[i,j])
        
        for i in range(r1_top,r1_bottom+1,2):
            for j in range(r1_left,r1_right+1,2):
                sigma = (p[i,j-1] + p[i,j+1])/dx**2 + (p[i-1,j] + p[i+1,j])/dy**2
                p[i,j] = p[i,j] + omega*((b[i,j] - sigma)/a - p[i,j])
                
        for i in range(r2_top,r2_bottom+1,2):
            for j in range(r2_left,r2_right+1,2):
                sigma = (p[i,j-1] + p[i,j+1])/dx**2 + (p[i-1,j] + p[i+1,j])/dy**2
                p[i,j] = p[i,j] + omega*((b[i,j] - sigma)/a - p[i,j])
        
        
        # Impose boundary conditions for pressure
        p[:,0] = p[:,1]
        p[:,nx-1] = p[:,nx-2]
        p[0,:] = p[1,:]
        p[ny-1,:] = 0.0  
        
        # Get residuals
        tmp = laplacian_2D(nx,ny,dx,dy,p)
        
        resd[1:ny-1,1:nx-1] = tmp[1:ny-1,1:nx-1] - b[1:ny-1,1:nx-1]

        # Frobenius norm
        nrm = np.sum(np.absolute(resd)**2)**0.5
        niter = niter + 1
        
    return(p,niter)

** (6) Visualize Results **

In [7]:
def visualize_2D(numframe,pressure,velocity_u,velocity_v,xx,yy):
    """
    Allows user to create gif of 2D system evolving in time 
    """
    
    outputfile = "NSE_2D_example.gif"
    delay_in_ms = 50
    
    fig,ax = plt.subplots(nrows = 1,ncols = 1,figsize = (10,10))
    cmap = plt.get_cmap('jet')
    
    def animate(frame):
        
        p = pressure[frame]
        u = velocity_u[frame]
        v = velocity_v[frame]
        ax.clear()
        plt.contourf(xx, yy, p, 40, cmap = cmap)
        ax.quiver(xx[::2, ::2], yy[::2, ::2], u[::2, ::2], v[::2, ::2], color='k')
        ax.axis('square')
        ax.set_xlabel('x',fontsize = 24)
        ax.set_ylabel('y',fontsize = 24)
        ax.set_title('Navier-Stokes Equations in 2D', fontsize = 24)
        
    anim = FuncAnimation(fig, animate, frames = (numframe), interval = delay_in_ms)
    return(anim)
    #anim.save(outputfile, writer='imagemagick', fps=30)

** Main Program **

In [8]:
# Discretization
nx = 50
ny = 50

# System size 
Lx = 2.0
Ly = 2.0
dx = Lx/(nx-1)
dy = Ly/(ny-1)

x = np.linspace(0,Lx,nx)
y = np.linspace(0,Ly,ny)

xx,yy = np.meshgrid(x,y)

# Timestep
dt = 0.001
t_0 = 0.0
t_f = 0.5

numframes = 100 # Number of frames to save for visualization (+1 for initial config)
interval = int((t_f - t_0)//(dt*numframes)) # Save every nth frame 
t_steps = interval*numframes # number of time steps

# Navier-Stokes parameters
rho = 1.0
nu = 0.1
tol = 0.001 # Pressure-poisson convergence tolerance

# Velocity field components
u = np.zeros((ny,nx))
v = np.zeros((ny,nx))

# Laplacians of velocity
lap_u = np.zeros((ny,nx))
lap_v = np.zeros((ny,nx))

# Pressure field, and RHS of pressure poisson equation
p = np.zeros((ny,nx))
b = np.zeros((ny,nx))

# Pressure gradients
dpdx = np.zeros((ny,nx))
dpdy = np.zeros((ny,nx))

# Advection terms 
advec_u = np.zeros((ny,nx))
advec_v = np.zeros((ny,nx))

# Pressure and velocity profiles 
time = np.zeros(numframes+1)
velocity_u = np.zeros((numframes+1,ny,nx))
velocity_v = np.zeros((numframes+1,ny,nx))
pressure = np.zeros((numframes+1,ny,nx))

# Initial configuration
t = t_0
time[0] = t
velocity_u[0,:,:] = u
velocity_v[0,:,:] = v
pressure[0,:,:] = p

for i in range(1,numframes):
    for j in range(0,interval):
        
        # Calculate advection terms
        advec_u = advec_terms_2D(nx,ny,dx,dy,u,v,u)
        advec_v = advec_terms_2D(nx,ny,dx,dy,u,v,v)
        
        # Calculate laplacians of velocity
        lap_u = laplacian_2D(nx,ny,dx,dy,u)
        lap_v = laplacian_2D(nx,ny,dx,dy,v)
        
        # Advance velocity field to time n+1/2
        u[1:ny-1,1:nx-1] = u[1:ny-1,1:nx-1] + dt*( -advec_u[1:ny-1,1:nx-1]  + nu*lap_u[1:ny-1,1:nx-1] )
        v[1:ny-1,1:nx-1] = v[1:ny-1,1:nx-1] + dt*( -advec_v[1:ny-1,1:nx-1]  + nu*lap_v[1:ny-1,1:nx-1] )
        
        # Solve poisson equation for pressure at time n+1/2
        b = RHS_poisson(nx,ny,dx,dy,rho,dt,u,v)
        p, niter = solve_poisson_2D(nx,ny,dx,dy,b,p,tol)
        
        # Calculate pressure gradients
        dpdx,dpdy = gradient_2D(nx,ny,dx,dy,p)
        
        # Advance velocity field to time n+1
        u[1:ny-1,1:nx-1] = u[1:ny-1,1:nx-1] - dt/rho*dpdx[1:ny-1,1:nx-1]
        v[1:ny-1,1:nx-1] = v[1:ny-1,1:nx-1] - dt/rho*dpdy[1:ny-1,1:nx-1]
        
        # Impose boundary conditions for u and v
        v[0,:] = 0.0
        v[ny-1,:] = 0.0
        v[:,0] = 0.0
        v[:,nx-1] = 0.0
    
        u[:,0] = 0.0
        u[:,nx-1] = 0.0
        u[0,:] = 0.0
        u[ny-1,:] = 1.0
        
        # Update time 
        t = t + dt
        
    # Save snapshot of system 
    time[i] = t
    velocity_u[i,:,:] = u
    velocity_v[i,:,:] = v
    pressure[i,:,:] = p

KeyboardInterrupt: 

In [ ]:
myanim = visualize_2D(numframes,pressure,velocity_u,velocity_v,xx,yy)
HTML(myanim.to_html5_video())